In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


In [2]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

In [3]:
from datasets import load_dataset
dataset = load_dataset("carblacac/twitter-sentiment-analysis")
text = dataset['train']['text']
feeling = dataset['train']['feeling']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

...

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(text, feeling, test_size=0.2, random_state=3113)
print(len(x_train), len(x_test))

95990 23998


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.84      0.77     11960
           1       0.81      0.68      0.74     12038

    accuracy                           0.76     23998
   macro avg       0.76      0.76      0.76     23998
weighted avg       0.76      0.76      0.76     23998



In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(), SVC())
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80     11960
           1       0.80      0.80      0.80     12038

    accuracy                           0.80     23998
   macro avg       0.80      0.80      0.80     23998
weighted avg       0.80      0.80      0.80     23998



In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

In [8]:
def preprocess(data):
  return {'text':data['text'],'feeling':data['feeling']}
train_data = dataset['train'].map(preprocess)
test_data = dataset['test'].map(preprocess)

Map:   0%|          | 0/119988 [00:00<?, ? examples/s]

Map:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [9]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
vocab_size = len(tokenizer.word_index)+1

In [10]:
train_seq = tokenizer.texts_to_sequences(train_data['text'])
test_seq = tokenizer.texts_to_sequences(test_data['text'])

In [11]:
max_len = 32
trainp = tf.keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=max_len)
testp = tf.keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=max_len)

In [12]:
embed_len = 32
model = Sequential([
    Embedding(input_dim=vocab_size,output_dim=embed_len,input_length=max_len),
    Bidirectional(LSTM(64,return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
import numpy
y_train = numpy.array(train_data['feeling'])
y_test = numpy.array(test_data['feeling'])
model.fit(trainp, y_train, epochs=4, validation_data=(testp,y_test))

Epoch 1/4
3750/3750 [==============================] - 489s 128ms/step - loss: 0.4746 - accuracy: 0.7729 - val_loss: 0.4438 - val_accuracy: 0.7925
Epoch 2/4
3750/3750 [==============================] - 464s 124ms/step - loss: 0.3107 - accuracy: 0.8692 - val_loss: 0.5132 - val_accuracy: 0.7740
Epoch 3/4
3750/3750 [==============================] - 466s 124ms/step - loss: 0.1921 - accuracy: 0.9246 - val_loss: 0.5760 - val_accuracy: 0.7685
Epoch 4/4
3750/3750 [==============================] - 464s 124ms/step - loss: 0.1302 - accuracy: 0.9488 - val_loss: 0.7182 - val_accuracy: 0.7636
